In [1]:
import cv2
import moviepy.editor as mpy
from IPython.display import clear_output, display

In [2]:
class RangeSelect:
    playerName = 'PlayerWindow'
    
    def __init__(self, videoFileName):
        self.readVideo = cv2.VideoCapture(videoFileName) #要抽取视频的文件名
        self.fps, self.frameSum = self.GetInformation()  #25.026, 200.0
        
    def GetInformation(self):
        return self.readVideo.get(cv2.CAP_PROP_FPS), int(self.readVideo.get(cv2.CAP_PROP_FRAME_COUNT))
    
    @staticmethod
    def onTrackbar(pos, pRangeSelect):
        pRangeSelect.readVideo.set(cv2.CAP_PROP_POS_FRAMES, pos)
        posTime = int( pRangeSelect.readVideo.get(cv2.CAP_PROP_POS_MSEC) )
        
        status, frame = pRangeSelect.readVideo.read()
        frame = cv2.putText(frame, str(posTime)+'ms', (16, 32), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0))
        cv2.imshow(pRangeSelect.playerName, frame)
    
    def SelectRange(self):
        cv2.namedWindow(self.playerName)
        cv2.createTrackbar('FramePos', RangeSelect.playerName , 0, self.frameSum-1, 
                           lambda pos: RangeSelect.onTrackbar(pos, self))
        RangeSelect.onTrackbar(0, self)

        cv2.waitKey(0)   #To exit the window, DON’T CLICK THE RED CROSS, just tap any key on your keyboard!
        cv2.destroyAllWindows()

In [3]:
class SliceVideo:
    def __init__(self, videoFileName):
        self.videoFileName = videoFileName #要抽取视频的文件名
        self.fps = 20
    
    def ExtractVideo(self, outputFileName):
        cap = cv2.VideoCapture(self.videoFileName)
        pos0 = int( input('起始帧号:') )
        cap.set(cv2.CAP_PROP_POS_FRAMES, pos0)
            
        status,frame = cap.read()
        self.fps = cap.get(cv2.CAP_PROP_FPS)
        videoWriter = cv2.VideoWriter( outputFileName, 
                              cv2.VideoWriter_fourcc(*'mp4v'), 
                              self.fps, 
                              (frame.shape[1],frame.shape[0]), 
                              True)
       
        pos1 = int( input('终止帧号:') )
        while status and pos0 < pos1:
            clear_output(wait=True)
            print(f"正在处理第{pos0}帧")    # use display(f) if you encounter performance issues
            videoWriter.write(frame)   #新文件大约是原先3倍，two-pass encode?
            status,frame = cap.read()
            pos0 = pos0 + 1

        videoWriter.release()
        cap.release()
        
    def ExtractAudio(self, outputFileName):
        pos0 = float( input('起始秒数:') )
        pos1 = float( input('终止秒数:') )
        audioBackground = mpy.AudioFileClip(self.videoFileName).subclip(pos0, pos1)
        audioBackground.write_audiofile(outputFileName)
    
    @staticmethod
    def VideoAddAudio(videoFileName, musicFileName, outputFileName, fps):
        origanVideo = mpy.VideoFileClip(videoFileName)
        audioBackground = mpy.AudioFileClip(musicFileName)
        finalVideo = origanVideo.set_audio(audioBackground)
        finalVideo.write_videofile(outputFileName, fps)

In [4]:
origalVideo = "video.mp4" #要抽取视频的文件名
fps = 25

task = '1'
while task != '0':
    task = input('0-exit, 1-选取范围(帧号和时间，只能按任意键退出)，2-存视频newSlice.mp4，3-存音频newSlice.mp3, 4-合成newSlice2.mp4:')
    if task == '1':
        bigVideo = RangeSelect(origalVideo)
        bigVideo.SelectRange()
        fps = bigVideo.fps
        #del bigVideo
    elif task == '2':
        v = SliceVideo(origalVideo)
        v.ExtractVideo('./newSlice.mp4')
    elif task == '3':
        v = SliceVideo(origalVideo)
        v.ExtractAudio('./newSlice.mp3')
    elif task == '4':
        SliceVideo.VideoAddAudio('newSlice.mp4', 'newSlice.mp3', 'newSlice2.mp4', fps)
    else:
        print(task)

正在处理第452帧
0-exit, 1-选取范围(帧号和时间，只能按任意键退出)，2-存视频newSlice.mp4，3-存音频newSlice.mp3, 4-合成newSlice2.mp4:3
起始秒数:10.4
终止秒数:18.1


chunk:   2%|█▏                                                               | 3/170 [00:00<00:07, 23.46it/s, now=None]

MoviePy - Writing audio in ./newSlice.mp3


MoviePy - Done.
0-exit, 1-选取范围(帧号和时间，只能按任意键退出)，2-存视频newSlice.mp4，3-存音频newSlice.mp3, 4-合成newSlice2.mp4:4


chunk:   1%|▊                                                                | 2/171 [00:00<00:08, 19.89it/s, now=None]

Moviepy - Building video newSlice2.mp4.
MoviePy - Writing audio in newSlice2TEMP_MPY_wvf_snd.mp3


t:   1%|▋                                                                    | 2/193 [00:00<00:09, 19.51it/s, now=None]

MoviePy - Done.
Moviepy - Writing video newSlice2.mp4



Moviepy - Done !
Moviepy - video ready newSlice2.mp4
0-exit, 1-选取范围(帧号和时间，只能按任意键退出)，2-存视频newSlice.mp4，3-存音频newSlice.mp3, 4-合成newSlice2.mp4:0
0
